#### Project CCF pySpark RDD

In [2]:
r=sc.parallelize([("A","B"),("B","C"),("B","D"),("D","E"),("F","G"),("G","H")],2)
r.collect()

Out[143]: [('A', 'B'), ('B', 'C'), ('B', 'D'), ('D', 'E'), ('F', 'G'), ('G', 'H')]

In [3]:
def CCF_Iterate_reduce(pair):
  key, values = pair[0],list(pair[1])
  #global accum
  min = key
  valueL = []
  for value in values:
    if value < min:
       min = value
    valueL.append(value)
  if min < key:
    yield((key, min))
    for value in valueL:
      if min != value:
        accum.add(1)
        yield((value, min))

In [4]:
log4jLogger = sc._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)

In [5]:

new_pair_flag = True
iteration = 0
accum = sc.accumulator(0)
dedupJob = r

In [6]:
#for secondary sort

from pyspark.rdd import portable_hash

n = 3 # number to hash
def partitioner(n):
   # Partition by the first item in the key tuple
    def partitioner_(x):
        return portable_hash(x[0]) % n
    return partitioner_


def CCF_Iterate_reduce_SS_shuffle(pair):
  key, values = pair[0],list(pair[1])
  min = values.pop(0)
  if min < key:
    yield((key, min))
    for value in values:
      accum.add(1)
      yield((value, min))

In [7]:
while new_pair_flag:
    iteration += 1
    newPair = False
    accum.value = 0

    # CCF-iterate (MAP)
    mapJob = dedupJob.flatMap(lambda e: (e,e[::-1]))
    #print(mapJob.collect())
    
    #Secondary Sort
    rddSS= (mapJob
  .keyBy(lambda kv: (kv[0], kv[1][0]))  # Create temporary composite key
  .repartitionAndSortWithinPartitions(numPartitions=n, partitionFunc=partitioner(n), ascending=True)
  .map(lambda x: x[1]))  # Drop key (note: there is no partitioner set anymore)
    
    # CCF-iterate (REDUCE)
    reduceJob = rddSS.groupByKey().flatMap(lambda pair: CCF_Iterate_reduce_SS_shuffle(pair))#.sortByKey()

    # CCF-dedup 
    dedupJob = reduceJob.distinct()

    # Force the RDD evalusation
    tmp = dedupJob.count()
    #print(dedupJob.collect())
    # Prepare next iteration
    #mapJob = dedupJob
    new_pair_flag = bool(accum.value)

    #LOGGER.warn("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag))
    print("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag) + str(accum.value))

[('B', 'A'), ('C', 'A'), ('D', 'A'), ('C', 'B'), ('H', 'F'), ('H', 'G'), ('E', 'D'), ('E', 'B'), ('D', 'B'), ('G', 'F')]
Iteration :1 --- newPair : True4
[('B', 'A'), ('C', 'A'), ('D', 'A'), ('E', 'A'), ('H', 'F'), ('E', 'B'), ('G', 'F'), ('D', 'B')]
Iteration :2 --- newPair : True9
[('B', 'A'), ('D', 'A'), ('E', 'A'), ('C', 'A'), ('H', 'F'), ('G', 'F')]
Iteration :3 --- newPair : True4
[('B', 'A'), ('C', 'A'), ('D', 'A'), ('H', 'F'), ('E', 'A'), ('G', 'F')]
Iteration :4 --- newPair : False0